## Using a planner

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.4.0-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.4.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.4.0-alpha"


Installed Packages Microsoft.SemanticKernel, 1.4.0 Microsoft.SemanticKernel.Connectors.OpenAI, 1.4.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.4.0-preview Microsoft.SemanticKernel.Plugins.Core, 1.4.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.4.0-alpha

In [2]:
#!import ../config/Settings.cs
#!import ../Plugins/EmailPlugin.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;
using System.IO;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();

kernel.Plugins.Clear();
var pluginPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "Plugins");
foreach (var plugin in new string[] { "WeatherPlugin", "WriterPlugin", "TranslatePlugin" })
{
    Console.WriteLine($"Loading ${pluginPath}${plugin}");
    kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginPath, plugin));
}
#pragma warning disable SKEXP0050
kernel.ImportPluginFromType<TimePlugin>();   
#pragma warning disable SKEXP0054
var bing = kernel.ImportPluginFromType<SearchUrlPlugin>("search");
// #pragma warning disable SKEXP0054
// var bingConnector = new BingConnector(bingApiKey);
// var bing = new WebSearchEnginePlugin(bingConnector);
// kernel.ImportPluginFromObject(bing, "bing");

var kernelPlugin = KernelPluginFactory.CreateFromFunctions(
    "AddressBookPlugin",
    "Manages properties of people with email addresses",
    new KernelFunction[] { 
            kernel.CreateFunctionFromMethod(
                ([Description("Person name")]string name) => $"{name}@contoso.com",
                functionName: "GetEmailAddress",
                description: "Given person name, return the person's email address"),
            kernel.CreateFunctionFromMethod(
                ([Description("Text to output")]string text) => Console.WriteLine(text),
                functionName: "DisplayText",
                description: "Output completion results to the screen")                
    }
);
kernel.Plugins.Add(kernelPlugin);


Loading $c:\Users\mrochon\source\repos\ai-samples\NotebookSK\4. PlanWithGraph\..\Plugins$WeatherPlugin
Loading $c:\Users\mrochon\source\repos\ai-samples\NotebookSK\4. PlanWithGraph\..\Plugins$WriterPlugin
Loading $c:\Users\mrochon\source\repos\ai-samples\NotebookSK\4. PlanWithGraph\..\Plugins$TranslatePlugin


List plugins and functions

In [3]:
foreach (KernelFunctionMetadata func in kernel.Plugins.GetFunctionsMetadata())
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"   {func.Name}: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("      Params:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}

Plugin: WeatherPlugin
   Simple: Create search url for a city
      Params:
      - city: 
        default: ''

Plugin: WriterPlugin
   Classification: Text classification
      Params:
      - input: 
        default: ''
      - label: 
        default: ''

Plugin: WriterPlugin
   Tips: Generates tips for a given topic
      Params:
      - input: 
        default: ''
      - language: 
        default: ''

Plugin: TranslatePlugin
   Basic: Translate from any language to English
      Params:
      - input: 
        default: ''

Plugin: TranslatePlugin
   MultiLanguage: Translate sentenses into a language of your choice
      Params:
      - language: 
        default: ''
      - input: 
        default: ''

Plugin: TimePlugin
   Date: Get the current date

Plugin: TimePlugin
   Today: Get the current date

Plugin: TimePlugin
   Now: Get the current date and time in the local time zone

Plugin: TimePlugin
   UtcNow: Get the current UTC date and time

Plugin: TimePlugin
   Time: Get th

Now try different prompts

In [6]:
#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });
//var plan = await planner.CreatePlanAsync(kernel, "Check the weather in Seattle, write an email, in Danish to Jane with dressing tips based on the results");
//var goal = "Check the weather in Seattle, find Jane's email address and write her an email with some dressing tips based on the results";    
//var goal = "How do I search for weather in Seattle?";
var goal = "Use web search to find Seattle weather";
var plan = await planner.CreatePlanAsync(kernel, goal);
Console.WriteLine("---Plan:\n");
Console.WriteLine(plan.ToString());
File.WriteAllText("plan.json", plan.ToString());

Console.WriteLine("---Result:\n");
var resp = await plan.InvokeAsync(kernel);
Console.WriteLine(resp);

// Should be able to save and re-load a plan: https://devblogs.microsoft.com/semantic-kernel/migrating-from-the-sequential-and-stepwise-planners-to-the-new-handlebars-and-stepwise-planner/

---Plan:

{{!-- Step 1: Define the search query --}}
{{set "query" "Seattle weather"}}

{{!-- Step 2: Call the search-BingSearchUrl helper with the query --}}
{{set "searchUrl" (search-BingSearchUrl query)}}

{{!-- Step 3: Output the search URL --}}
{{json searchUrl}}
---Result:

https://www.bing.com/search?q=Seattle%20weather


You can now modify, load and execute the plan.

In [12]:
var plan2Source = File.ReadAllText("plan.json");
plan2Source = plan2Source.Replace("Seattle", "Chicago");
#pragma warning disable SKEXP0060
var plan2 = new HandlebarsPlan(plan2Source);
var resp2 = await plan2.InvokeAsync(kernel);
Console.WriteLine(resp2);

https://www.bing.com/search?q=Chicago%20weather
